# A full business solution

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.


In [2]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
import ollama

In [3]:
client = ollama.Client()

In [4]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [5]:
ed = Website("https://facebook.com")
ed.links

['//www.facebook.com/',
 '//www.facebook.com/',
 '//www.facebook.com/help/?ref=href052']

## First step: Have llama3.2 figure out which links are relevant

### Use a call to llama3.2 to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

In [6]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [7]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}



In [8]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [9]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://facebook.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
//www.facebook.com/
//www.facebook.com/
//www.facebook.com/help/?ref=href052


In [10]:
def get_links(url):
    website = Website(url)
    response = client.chat(
        model="llama3.2",
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
    )
    result = response["message"]["content"]
    try:
        data = json.loads(result)  # attempt to parse JSON
    except json.JSONDecodeError:
        # fallback if model returned extra text
        result = result[result.find("{"):result.rfind("}")+1]  # extract JSON-like part
        data = json.loads(result)

    return data 

In [11]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
# huggingface.links

In [12]:
get_links("https://huggingface.co")

{'links': [{'type': 'About page', 'url': 'https://huggingface.co/'},
  {'type': 'Company page', 'url': 'https://huggingface.co/brand'},
  {'type': 'Changelog page', 'url': 'https://huggingface.co/changelog'},
  {'type': 'Blog page', 'url': 'https://blog.huggingface.co/'},
  {'type': 'GitHub repository', 'url': 'https://github.com/huggingface'},
  {'type': 'Twitter account', 'url': 'https://twitter.com/huggingface'},
  {'type': 'LinkedIn company page',
   'url': 'https://www.linkedin.com/company/huggingface/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to llama3.2

In [13]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [15]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [16]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [17]:
get_brochure_user_prompt("Facebook", "https://facebook.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.facebook.com/about/'}, {'type': 'company page', 'url': 'https://www.facebook.com/'}]}


"You are looking at a company called: Facebook\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nError\nWebpage Contents:\nSorry, something went wrong.\nWe're working on getting this fixed as soon as we can.\nGo back\nMeta © 2025 ·\nHelp\n\n\n\nabout page\nWebpage Title:\nError\nWebpage Contents:\nSorry, something went wrong.\nWe're working on getting this fixed as soon as we can.\nGo back\nMeta © 2025 ·\nHelp\n\n\n\ncompany page\nWebpage Title:\nError\nWebpage Contents:\nSorry, something went wrong.\nWe're working on getting this fixed as soon as we can.\nGo back\nMeta © 2025 ·\nHelp\n\n"

In [18]:
def create_brochure(company_name, url):
    response = ollama.chat(
        model="llama3.2",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response["message"]["content"]
    display(Markdown(result))

In [19]:
create_brochure("Facebook", "https://facebook.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.facebook.com/'}, {'type': 'company/careers page', 'url': 'https://www.facebook.com/careers/'}]}


**Facebook Brochure**

**Company Overview**
===============

Facebook is a leading social media and technology company that connects people, businesses, and organizations around the world.

**Our Mission**
-------------

To give people the power to build community and bring the world closer together.

**Our Culture**
-------------

At Facebook, we're committed to building a culture of inclusivity, diversity, and respect. We believe in fostering an environment where everyone feels valued and empowered to be their authentic selves. Our team is passionate about creating innovative products and services that make a positive impact on people's lives.

* **Collaborative Environment**: We encourage teamwork, creativity, and open communication.
* **Innovation Hub**: We invest in emerging technologies and trends to stay ahead of the curve.
* **Global Reach**: We connect people across the globe, fostering global understanding and collaboration.

**Our Customers**
----------------

Facebook's platform is used by over 2.7 billion monthly active users, making it one of the most widely used social media platforms in the world. Our customers come from diverse backgrounds and industries, including:

* Individuals
* Small businesses
* Large enterprises
* Non-profit organizations

**Careers at Facebook**
----------------------

Are you passionate about technology, innovation, and community-building? Join our team of talented individuals who share your values and are committed to making a positive impact.

* **Open Positions**: Check our careers page for current job openings in various fields.
* **Benefits Package**: We offer competitive salaries, comprehensive benefits, and opportunities for growth and development.

**Investor Information**
----------------------

For investors interested in learning more about Facebook's financial performance, business strategy, and future plans:

* **SEC Filings**: Visit the Securities and Exchange Commission (SEC) website for our latest quarterly and annual reports.
* **Financial Highlights**: Explore our financial highlights, including revenue growth, operating expenses, and cash flow.

**Get in Touch**
----------------

Stay up-to-date with Facebook's latest news, updates, and announcements:

* **Contact Us**: Reach out to our customer support team via phone, email, or social media channels.
* **Follow Us**: Stay connected on Facebook, Twitter, Instagram, and other social media platforms.

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>